In [ ]:
# PIPELINE SIMULACIÓN DE REPERTORIOS + ANALISIS DE MÈTRICAS DE EXPANSIÓN CLONAL, DIVERSIDAD Y SHM.IMMUNESIM/IMMCANTATION
# Este pipeline permite generar repertorios inmunes simulados con parámetros controlados 
# y analizarlos mediante métricas de expansión clonal, diversidad y mutación somática (SHM).
# Integra herramientas de simulación como ImmuneSim y módulos del ecosistema Immcantation (Change-O, alakazam, shazam)
# para el procesamiento y análisis bioinformático. La ejecución se distribuye entre scripts en R y Python, 
# coordinados mediante un script Bash (.sh), lo que garantiza automatización, reproducibilidad y trazabilidad en cada etapa del flujo. 
# Además, se utiliza un notebook interactivo (.ipynb) para ejecutar y documentar los análisis en R, 
# mientras que el script Bash (.sh) gestiona la ejecución de módulos en Python y la orquestación general del pipeline.

In [ ]:

# Instalar paquete renv paquete de R que sirve para manejar entornos reproducibles de proyectos. Su función principal es aislar y “congelar” las versiones de los paquetes que usas en un proyecto.
# install.packages("renv")
# library(renv)
# renv::init() Inicializar proyecto renv generando renv.lock
# Paquetes CRAN install.packages(c("dplyr", "ggplot2", "viridisLite", "readr", "here"))
# Paquetes Bioconductor (por ejemplo para Biostrings, IRanges, GenomicRanges)
# if (!requireNamespace("BiocManager", quietly = TRUE))
#   install.packages("BiocManager")
# BiocManager::install(c("Biostrings", "IRanges", "GenomicRanges"))
# renv::snapshot() actualiza el lockfile con las versiones exactas usadas
# Para restaurar en otro computador 
# renv::restore()




In [2]:
# pak::pkg_install("BiocManager")
# BiocManager::install("Biostrings")
# pak::pkg_install("immuneSIM")
library(Biostrings)
library(immuneSIM)
library(here)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: 'S4Vectors'


The follow

In [ ]:
# SIMULACION DE REPERTORIOS CON IMMUNESIM
# Descripción de variables:
# name_repertoire = nombre del repertorio
# number_of_seqs = numero de secuencias 
# species = especie
# receptor = ig (bcr)o tr (tcr)
# chain = cadena pesada o liviana
# verbose = activación de mensajes en la consola durante la simulación.
# equal_cc = distribución de recuento de clone, si todos los clonotipos tienen o no mismo tamaño
# user_defined_alpha = uniformidad de la distribucion, valor mas alto mas desiguales los clones. 
# shm = mutaciones somaticas basadas en modelos estadisticos que definen como se distribuyen las mutaciones en las secuencias simuladas.
# none= no se simulan mutaciones / poisson=mutaciones distribuidas aleatoriamente segun modelo poisson, sin sesgo y constante en la secuencia/data= modelo basado en perfiles reales, mutaciones se concentran en CDR, baja aleatoriedad/ naive= simula secuencias sin SHM, pero puede incluir artefactos técnicos/ motif= mutaciones dirigidas por motivos especificos, introduce sesgos basados en hotspots conocidos.
# shm.prob = probabilidad de que ocurra el evento 15/350 son 15 eventos de mutaciones en una secuencia de 350 nucleotidos.
# vdj_noise = ruido que se genera porque se modifica la probabilidad con la que se seleccionan ciertos genes V, D o J durante la simulación de recombinación V(D)J. va de 0-1.
# max_cdr3_length = 100, min_cdr3_length= controlan el umbral máximo y mínimo para la longitud del aminoácido CDR3. En humanos, la longitud típica de CDR3 en IgH suele estar entre 10 y 25 aminoácidos, con una mediana alrededor de 15–16.


In [3]:

# SIMULACIÓN ESCENARIO 1
# Cargar librerías

# Parámetro: número de secuencias

n_seqs <- 10  # Cambiar a 10, 100, 1000, etc.

# Simular repertorio A naive sin expansión y sin SHM en R

sim_repertoire <- immuneSIM(
  name_repertoire = paste0("sim_rep_", n_seqs),
  number_of_seqs  = n_seqs,
  species         = "hs",
  receptor        = "ig",
  chain           = "h",
  verbose         = TRUE,
  equal_cc        = TRUE,
  shm             = "none",
  shm.prob        = 0,
  vdj_noise       = 0,
  max_cdr3_length = 15,
  min_cdr3_length = 15
)


# Preparar secuencias para FASTA

dna_sequences <- Biostrings::DNAStringSet(sim_repertoire$sequence)
names(dna_sequences) <- paste0("seq", seq_along(dna_sequences))


# Crear carpeta 'data/input' si no existe

# dir.create(here("data", "input"), recursive = TRUE, showWarnings = FALSE)


# Guardar archivo FASTA en 'data/input'

out_file <- here("..", "data", "input",
                 paste0("repertorio_A_insilico_", n_seqs, "_seqs.fasta"))

writeXStringSet(dna_sequences, filepath = out_file)
message("Archivo guardado en: ", out_file)



initializing sim..
simulated sequences: 10 


Archivo guardado en: /Users/catg/Desktop/SOFIAC/Gitsofia/tesisbioinf-sofia/notebooks/../data/input/repertorio_A_insilico_10_seqs.fasta



In [ ]:

out_file <- here("..", "data", "input",
                 paste0("repertorio_A_insilico_", n_seqs, "_seqs.fasta"))
out_file              

[1] "/Users/catg/Desktop/SOFIA C/Gitsofia/tesisbioinf-sofia/notebooks/../data/input/repertorio_A_insilico_10_seqs.fasta"

In [ ]:
here()

[1] "/Users/catg/Desktop/SOFIA C/Gitsofia/tesisbioinf-sofia/notebooks"

In [ ]:
# SIMULACIÓN ESCENARIO 2
# Cargar librerías


# Parámetro: número de secuencias

n_seqs <- 10  # Cambiar a 10, 100, 1000, etc.

# Simular repertorio  B pre-B naive con expansión y  SHM en R

sim_repertoire <- immuneSIM(
  name_repertoire = paste0("sim_rep_", n_seqs),
 number_of_seqs = 10,
  species = "hs",
  receptor = "ig",
  chain = "h",
  verbose= TRUE,
  equal_cc = FALSE,
  user_defined_alpha = 1.0,  
  shm = "poisson",
  shm.prob = 05/350,
  vdj_noise = 0.1,
  max_cdr3_length =18, 
  min_cdr3_length =15,
 
)


# Preparar secuencias para FASTA

dna_sequences <- Biostrings::DNAStringSet(sim_repertoire$sequence)
names(dna_sequences) <- paste0("seq", seq_along(dna_sequences))


# Crear carpeta 'data/input' si no existe

# dir.create(here("data", "input"), recursive = TRUE, showWarnings = FALSE)


# Guardar archivo FASTA en 'data/input'

out_file <- here("..", "data", "input",
                 paste0("repertorio_B_insilico_", n_seqs, "_seqs.fasta"))

writeXStringSet(dna_sequences, filepath = out_file)
message("Archivo guardado en: ", out_file)




Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: 'S4Vectors'


The follow

initializing sim..
simulated sequences: 10 


Archivo guardado en: /Users/catg/Desktop/SOFIAC/Gitsofia/tesisbioinf-sofia/notebooks/../data/input/repertorio_B_insilico_10_seqs.fasta



In [ ]:
# SIMULACIÓN ESCENARIO 3
# Cargar librerías


# Parámetro: número de secuencias

n_seqs <- 10  # Cambiar a 10, 100, 1000, etc.

# Simular repertorio C mixto sangre periferica con expansión y  SHM en R

sim_repertoire <- immuneSIM(
  name_repertoire = paste0("sim_rep_", n_seqs),
 number_of_seqs = 10,
  species = "hs",
  receptor = "ig",
  chain = "h",
  verbose= TRUE,
  equal_cc = FALSE,
  user_defined_alpha = 0.5,  
  shm = "data",
  shm.prob = 15/350,
  vdj_noise = 0.25,
  max_cdr3_length =22, 
  min_cdr3_length =15,
 
)

# Preparar secuencias para FASTA

dna_sequences <- Biostrings::DNAStringSet(sim_repertoire$sequence)
names(dna_sequences) <- paste0("seq", seq_along(dna_sequences))


# Crear carpeta 'data/input' si no existe

# dir.create(here("data", "input"), recursive = TRUE, showWarnings = FALSE)


# Guardar archivo FASTA en 'data/input'

out_file <- here("..", "data", "input",
                 paste0("repertorio_C_insilico_", n_seqs, "_seqs.fasta"))

writeXStringSet(dna_sequences, filepath = out_file)
message("Archivo guardado en: ", out_file)





Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: 'S4Vectors'


The follow

initializing sim..
simulated sequences: 10 


Archivo guardado en: /Users/catg/Desktop/SOFIAC/Gitsofia/tesisbioinf-sofia/notebooks/../data/input/repertorio_C_insilico_10_seqs.fasta

